In [ ]:
from mpi4py import MPI

from basix.ufl import element, mixed_element

import dolfinx
from dolfinx.mesh import CellType, create_unit_square

from dolfinx.fem import (
    Constant,
    Function,
    dirichletbc,
    extract_function_spaces,
    form,
    functionspace,
    locate_dofs_topological,
)


In [ ]:
msh = create_unit_square(MPI.COMM_WORLD, 96, 96, CellType.triangle)

P1 = element("Lagrange", msh.basix_cell(), degree=1)
E = mixed_element([P1, P1, P1])
V = functionspace(msh, E)